### <em>Natural Language Processing for the Creative Industries</em>
# MINI-PROJECT: GPT2-Proc-Corpora-Generator-2
_________________

### 5. Required Libraries & Utilities

In [11]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
!pip install transformers==4.3.3
from torch.nn import CrossEntropyLoss
import torch.nn as nn
#from transformers.models.bert.modeling_bert import BertModel, BertForMaskedLM
#from transformers.modeling_gpt2 import GPT2PreTrainedModel, GPT2Model
from transformers.models.gpt2.modeling_gpt2 import GPT2PreTrainedModel, GPT2Model
import torch
import json
import os
from tqdm import tqdm
from torch.utils.data import Dataset
import logging
from torch.nn.utils.rnn import pad_sequence
import os
import random
import numpy as np
import argparse
import logging
from transformers.models.gpt2.modeling_gpt2 import GPT2Config
from transformers import BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import copy


logger = logging.getLogger(__name__)



Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.9 MB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 59.9 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [12]:
from torch.nn import CrossEntropyLoss
import torch.nn as nn
from transformers.modeling_gpt2 import GPT2PreTrainedModel, GPT2Model
import torch
import json
import os
from tqdm import tqdm
from torch.utils.data import Dataset
import logging
from torch.nn.utils.rnn import pad_sequence
import os
import random
import numpy as np
import argparse
import logging
from transformers.modeling_gpt2 import GPT2Config
from transformers import BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import copy


logger = logging.getLogger(__name__)

### 6. Define and Processing Model Data

In [13]:
#Adapted from Liucongg- https://github.com/liucongg/GPT2-NewsTitle
#Define the data required for the model

class GPT2ProductDataSet(Dataset):
    def __init__(self,
                 tokenizer,
                 max_len,
                 title_max_len,
                 data_dir,
                 data_set_name,
                 path_file=None,
                 is_overwrite=False):

        self.tokenizer = tokenizer
        self.content_id = self.tokenizer.convert_tokens_to_ids("[Content]")
        self.title_id = self.tokenizer.convert_tokens_to_ids("[Title]")

        self.space_id = self.tokenizer.convert_tokens_to_ids("[Space]")
        self.max_len = max_len
        self.title_max_len = title_max_len
        cached_feature_file = os.path.join(
            data_dir, "cached_{}_{}".format(data_set_name, max_len))

        if os.path.exists(cached_feature_file) and not is_overwrite:
            logger.info("There is already a cache file {}, load it directly".format(cached_feature_file))
            self.data_set = torch.load(cached_feature_file)["data_set"]
        else:
            logger.info("No cache file {}, data preprocessing operation".format(cached_feature_file))
            self.data_set = self.load_data(path_file)
            logger.info(
                "After the data preprocessing operation is completed, save the processed data in {} as a cache file".format(cached_feature_file))
            torch.save({"data_set": self.data_set}, cached_feature_file)

    #Load the original data and generate processed data
    def load_data(self, path_file):
        self.data_set = []
        with open(path_file, "r", encoding="utf-8") as fh:
            for line in tqdm(fh.readlines(), desc="iter",disable=False):
                line = line.strip()
                if not line:
                    continue
                sample = json.loads(line)
                # Indexing titles and contents
                input_ids, token_type_ids = self.convert_feature(sample)
                self.data_set.append({
                    "input_ids": input_ids,
                    "token_type_ids": token_type_ids
                })
        return self.data_set

    #Because of my generation goal
    #sample: a dictionary containing product titles and descriptions, the format is changed to {"content": title, "title": content}
    def convert_feature(self, sample):
        input_ids = []
        token_type_ids = []
        
        #Because the original model generates "Title" from "Content", the path is changed here to adapt to my data
        content_tokens = self.tokenizer.tokenize(sample["title"]) 
        title_tokens = self.tokenizer.tokenize(sample["content"].replace(
            " ", "[Space]"))

        if len(title_tokens) > self.title_max_len:
            title_tokens = title_tokens[:self.title_max_len]

        if len(content_tokens) > self.max_len - len(title_tokens) - 3:
            content_tokens = content_tokens[:self.max_len - len(title_tokens) -
                                            3]
        #input_ids & token_type_ids
        input_ids.append(self.tokenizer.cls_token_id)
        token_type_ids.append(self.content_id)
        input_ids.extend(self.tokenizer.convert_tokens_to_ids(content_tokens))
        token_type_ids.extend([self.content_id] * len(content_tokens))
        input_ids.append(self.tokenizer.sep_token_id)
        token_type_ids.append(self.content_id)
        input_ids.extend(self.tokenizer.convert_tokens_to_ids(title_tokens))
        token_type_ids.extend([self.title_id] * len(title_tokens))
        input_ids.append(self.tokenizer.sep_token_id)
        token_type_ids.append(self.title_id)

        assert len(input_ids) == len(token_type_ids)
       #assert len(input_ids) <= self.max_len
        return input_ids, token_type_ids

    def __len__(self):
        return len(self.data_set)

    def __getitem__(self, idx):
        instance = self.data_set[idx]
        return instance

#Data processing into tensor form    
def collate_func(batch_data):
    batch_size = len(batch_data)
    if batch_size == 0:
        return {}
    input_ids_list, token_type_ids_list = [], []
    for instance in batch_data:
        input_ids_temp = instance["input_ids"]
        token_type_ids_temp = instance["token_type_ids"]
        input_ids_list.append(torch.tensor(input_ids_temp, dtype=torch.long))
        token_type_ids_list.append(
            torch.tensor(token_type_ids_temp, dtype=torch.long))
    return {
        "input_ids":
        pad_sequence(input_ids_list, batch_first=True, padding_value=0),
        "token_type_ids":
        pad_sequence(token_type_ids_list, batch_first=True, padding_value=0)
    }

### 7.Rewrite GPT2LMHeadModel

In [14]:
#Adapted from Liucongg- https://github.com/liucongg/GPT2-NewsTitle
#Modify the calculation loss part

class GPT2LMHeadModel(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.init_weights()

    def forward(self,
                input_ids=None,
                past=None,
                token_type_ids=None,
                labels=None,
                title_id=None):

        # Get the output result of the GPT2 model
        transformer_outputs = self.transformer(input_ids,
                                               past=past,
                                               token_type_ids=token_type_ids)
        #size:[batch_size, sequence_length, config.n_embd]
        hidden_states = transformer_outputs[0]
        #size:[batch_size, sequence_length, config.vocab_size]
        lm_logits = self.lm_head(hidden_states)
        outputs = (lm_logits, ) + transformer_outputs[1:]

        if labels is not None:
            if title_id is None or token_type_ids is None:
                raise Exception(
                    "When labels is not None, neither ‘title_id’ nor ‘token_type_ids’ can be None。")

            # size:[batch_size, sequence_length]
            mask = (token_type_ids == title_id).long()
            # size:[batch_size, sequence_length]
            labels = labels * mask
            # Calculate the loss of sequence_length-1 tokens
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss_fct = CrossEntropyLoss(ignore_index=0, reduction="sum")
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)),
                            shift_labels.view(-1))
            num = shift_labels.ne(0).long().sum().item()
            loss = loss / num
            outputs = (loss, ) + outputs
        return outputs  # (loss), lm_logits, presents, (all hidden_states), (attentions)

### 8.Training model

In [15]:
#Adapted from Liucongg- https://github.com/liucongg/GPT2-NewsTitle
#Training of the GPT2 model for generating product descriptions from product titles

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    level=logging.INFO)
logger = logging.getLogger(__name__)


def train(model, device, train_data, test_data, args, tokenizer):
    tb_write = SummaryWriter()
    if args.gradient_accumulation_steps < 1:
        raise ValueError("The gradient_accumulation_steps parameter is invalid and must be greater than or equal to 1")
    train_batch_size = int(args.train_batch_size /
                           args.gradient_accumulation_steps)
    train_sampler = RandomSampler(train_data)
    train_data_loader = DataLoader(train_data,
                                   sampler=train_sampler,
                                   batch_size=train_batch_size,
                                   collate_fn=collate_func)
    total_steps = int(
        len(train_data_loader) * args.num_train_epochs /
        args.gradient_accumulation_steps)
    logger.info("The total number of training steps is: ()".format(total_steps))
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [{
        'params':
        [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay':
        0.01
    }, {
        'params':
        [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay':
        0.0
    }]

    optimizer = AdamW(optimizer_grouped_parameters,
                      lr=args.learning_rate,
                      eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(args.warmup_proportion * total_steps),
        num_training_steps=total_steps)

    torch.cuda.empty_cache()
    model.train()
    title_id = train_data.title_id
    tr_loss, logging_loss, min_loss = 0.0, 0.0, 0.0
    global_step = 0
 
    for iepoch in trange(0,
                         int(args.num_train_epochs),
                         desc="Epoch",
                         disable=False):
        iter_bar = tqdm(train_data_loader,
                        desc="Iter (loss=X.XXX)",
                        disable=False)
        for step, batch in enumerate(iter_bar):
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            outputs = model.forward(input_ids=input_ids,
                                    token_type_ids=token_type_ids,
                                    labels=input_ids,
                                    title_id=title_id)
            loss = outputs[0]
            tr_loss += loss.item()
            iter_bar.set_description("Iter (loss=%5.3f)" % loss.item())
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                           args.max_grad_norm)
 
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    tb_write.add_scalar("lr",
                                        scheduler.get_lr()[0], global_step)
                    tb_write.add_scalar("train_loss",
                                        (tr_loss - logging_loss) /
                                        (args.logging_steps *
                                         args.gradient_accumulation_steps),
                                        global_step)
                    logging_loss = tr_loss

                if args.eval_steps > 0 and global_step % args.eval_steps == 0:
                    eval_loss = evaluate(model, device, test_data, args)
                    tb_write.add_scalar("test_loss", eval_loss, global_step)
                    model.train()

        output_dir = os.path.join(args.output_dir,
                                  "checkpoint-{}".format(global_step))
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        torch.cuda.empty_cache()

#Model testing on the test dataset
def evaluate(model, device, test_data, args):

    # Construct the 'DataLoader'of the test set
    test_sampler = SequentialSampler(test_data)
    test_data_loader = DataLoader(test_data,
                                  sampler=test_sampler,
                                  batch_size=args.test_batch_size,
                                  collate_fn=collate_func)
    iter_bar = tqdm(test_data_loader, desc="iter", disable=False)
    title_id = test_data.title_id
    total_loss, total = 0.0, 0.0

    for step, batch in enumerate(iter_bar):
        model.eval()
        with torch.no_grad():
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            outputs = model.forward(input_ids=input_ids,
                                    token_type_ids=token_type_ids,
                                    labels=input_ids,
                                    title_id=title_id)
            loss = outputs[0]
            loss = loss.item()
            total_loss += loss * len(batch["input_ids"])
            total += len(batch["input_ids"])

    test_loss = total_loss / total
    return test_loss

#Set training model parameters
def set_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device',
                        default='0',
                        type=str,
                        help='Set the graphics card used for training or testing')
    parser.add_argument('--config_path',
                        default='config/config.json',
                        type=str,
                        help='Model parameter configuration information')
    parser.add_argument('--vocab_path',
                        default='vocab/vocab.txt',
                        type=str,
                        help='vocab')
#I put the data of 1/25 data set and 1/200 data set in 'data_dir', which is convenient for replacing files and training models with different amounts of data.
    parser.add_argument('--train_file_path',
                        default='data_dir/train.json', #Replaceable
                        type=str)
    parser.add_argument('--test_file_path',
                        default='data_dir/test.json',#Replaceable
                        type=str)
    
    parser.add_argument('--pretrained_model_path',
                        default=None,
                        type=str)
    parser.add_argument('--data_dir',
                        default='data_dir',
                        type=str)
    parser.add_argument('--num_train_epochs',
                        default=50, #The number of train epochs this time equals 20
                        type=int)
    parser.add_argument('--train_batch_size',
                        default=8, #Set the biaze size of train
                        type=int)
    parser.add_argument('--test_batch_size',#Set the biaze size of test
                        default=8,
                        type=int)
    parser.add_argument('--learning_rate',
                        default=1e-4,
                        type=float)
    parser.add_argument('--warmup_proportion',
                        default=0.1,
                        type=float)
    parser.add_argument('--adam_epsilon',
                        default=1e-8,
                        type=float)
    parser.add_argument('--logging_steps',
                        default=20,
                        type=int)
    parser.add_argument('--eval_steps',
                        default=4000,
                        type=int)
    parser.add_argument('--gradient_accumulation_steps',
                        default=1,
                        type=int)
    parser.add_argument('--max_grad_norm', default=1.0, type=float)
    parser.add_argument('--output_dir',
                        default='output_dir/',
                        type=str)
    parser.add_argument('--seed', type=int, default=2020)
    parser.add_argument('--max_len',
                        type=int,
                        default=150)
    parser.add_argument('--title_max_len',
                        type=int,
                        default=85)
    
    # According to my dataset, set the length of the input text
    return parser.parse_args(['--max_len', '20', '--title_max_len', '88'])


def main():
    args = set_args()
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICE"] = args.device
    device = torch.device("cuda" if torch.cuda.is_available()
                          and int(args.device) >= 0 else "cpu")

    if args.seed:
        torch.manual_seed(args.seed)
        random.seed(args.seed)
        np.random.seed(args.seed)

    model_config = GPT2Config.from_json_file(args.config_path)

    if args.pretrained_model_path:
        model = GPT2LMHeadModel.from_pretrained(args.pretrained_model_path)
    else:
        model = GPT2LMHeadModel(config=model_config)

    tokenizer = BertTokenizer.from_pretrained(args.vocab_path,
                                              do_lower_case=True)

    tokenizer.save_pretrained(args.output_dir)
    tokenizer.add_tokens("[Space]", special_tokens=True)
    
    # Create the output directory of the model
    if not os.path.exists(args.output_dir):
        os.mkdir(args.output_dir)
    # Load train data and test data
    train_data = GPT2ProductDataSet(tokenizer, args.max_len,
                                      args.title_max_len, args.data_dir,
                                      "train", args.train_file_path)
    test_data = GPT2ProductDataSet(tokenizer, args.max_len,
                                     args.title_max_len, args.data_dir, "test",
                                     args.test_file_path)

    train(model, device, train_data, test_data, args, tokenizer)

**Note:** The model has already been collected and saved several checkpoints. Therefore this under cell need not be activated.

In [ ]:
# Start training(Don't run this)
main()

10/04/2022 07:42:13 - INFO - transformers.tokenization_utils_base -   Model name 'vocab/vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'vocab/vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
10/04/2022 07:42:13 - WARNING - transformers.tokenization_utils_base -   Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
10/04/2022 07:42:13 - I

### 9.Generating corpora from new model

In [6]:
!pip install sacrebleu --user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [16]:
#Adapted from Liucongg- https://github.com/liucongg/GPT2-NewsTitle
#Generate product descriptions by entering product titles

import torch
import os
import argparse
from transformers import BertTokenizer
import torch.nn.functional as F
import copy
import sacrebleu
import json

def set_args_generate():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device',
                        default='0',
                        type=str)
    #Set the checkpoint of the loaded model(11184),You can change the checkponit for comparison
    parser.add_argument('--model_path',
                        default='output_dir/checkpoint-99950',#Replaceable
                        type=str)
    parser.add_argument('--vocab_path',
                        default='vocab/vocab.txt',
                        type=str)
    parser.add_argument('--batch_size', default=3, type=int)
    parser.add_argument('--generate_max_len',
                        default=32,
                        type=int)
    parser.add_argument('--repetition_penalty',
                        default=1.2,
                        type=float)
    parser.add_argument('--top_k',
                        default=10,
                        type=float)
    parser.add_argument('--top_p',
                        default=0.95,
                        type=float)
    parser.add_argument('--max_len',
                        type=int,
                        default=512)
    
    # According to my dataset, set the length of the input text
    return parser.parse_args(['--max_len', '85', '--generate_max_len','150'])


def top_k_top_p_filtering(logits, top_k, top_p, filter_value=-float("Inf")):
    assert logits.dim() == 2
    top_k = min(top_k, logits[0].size(-1))

    if top_k > 0:
        for logit in logits:
            indices_to_remove = logit < torch.topk(logit, top_k)[0][..., -1,
                                                                    None]
            logit[indices_to_remove] = filter_value
            
    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits,
                                                   descending=True,
                                                   dim=-1)
        
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1),
                                        dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
            ..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        for index, logit in enumerate(logits):
            indices_to_remove = sorted_indices[index][
                sorted_indices_to_remove[index]]
            logit[indices_to_remove] = filter_value
    return logits

#Make predictions on a single sample
def predict_one_sample(model, tokenizer, device, args, content):

    # Since the above has been modified, here is the preprocessing of the product title
    content_tokens = tokenizer.tokenize(content)
    if len(content_tokens) > args.max_len - 3 - args.generate_max_len:
        content_tokens = content_tokens[:args.max_len - 3 -
                                        args.generate_max_len]

    content_id = tokenizer.convert_tokens_to_ids("[Content]")
    title_id = tokenizer.convert_tokens_to_ids("[Title]")
    unk_id = tokenizer.convert_tokens_to_ids("[UNK]")
    sep_id = tokenizer.convert_tokens_to_ids("[SEP]")
    content_tokens = ["[CLS]"] + content_tokens + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(content_tokens)
    input_ids = [copy.deepcopy(input_ids) for _ in range(args.batch_size)]
    token_type_ids = [[content_id] * len(content_tokens)
                      for _ in range(args.batch_size)]
    input_tensors = torch.tensor(input_ids).long().to(device)
    token_type_tensors = torch.tensor(token_type_ids).long().to(device)
    next_token_type = torch.tensor([[title_id] for _ in range(args.batch_size)
                                    ]).long().to(device)

    generated = []
    finish_set = set()
    with torch.no_grad():
        #Since the above has been modified, here is traverse the maximum length of the generated content.
        for _ in range(args.generate_max_len):
            outputs = model(input_ids=input_tensors,
                            token_type_ids=token_type_tensors)
            next_token_logits = outputs[0][:, -1, :]
            
            for index in range(args.batch_size):
                for token_id in set(
                    [token_ids[index] for token_ids in generated]):
                    next_token_logits[index][
                        token_id] /= args.repetition_penalty
   
            for next_token_logit in next_token_logits:
                next_token_logit[unk_id] = -float("Inf")

            filter_logits = top_k_top_p_filtering(next_token_logits,
                                                  top_k=args.top_k,
                                                  top_p=args.top_p)
            next_tokens = torch.multinomial(F.softmax(filter_logits, dim=-1),
                                            num_samples=1)

            for index, token_id in enumerate(next_tokens[:, 0]):
                if token_id == sep_id:
                    finish_set.add(index)
            finish_flag = True
            
            for index in range(args.batch_size):
                if index not in finish_set:
                    finish_flag = False
                    break
            if finish_flag:
                break

            generated.append([token.item() for token in next_tokens[:, 0]])
            input_tensors = torch.cat((input_tensors, next_tokens), dim=-1)
            token_type_tensors = torch.cat(
                (token_type_tensors, next_token_type), dim=-1)

        candidate_responses = []
 
        for index in range(args.batch_size):
            responses = []
            for token_index in range(len(generated)):
                if generated[token_index][index] != sep_id:
                    responses.append(generated[token_index][index])
                else:
                    break
            candidate_responses.append("".join(
                tokenizer.convert_ids_to_tokens(responses)).replace(
                    "##", "").replace("[space]", " "))
    return candidate_responses

#Generated main function
def main_generate():
    args = set_args_generate()

    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICE"] = args.device
    device = torch.device("cuda" if torch.cuda.is_available()
                          and int(args.device) >= 0 else "cpu")

    tokenizer = BertTokenizer.from_pretrained(args.vocab_path,
                                              do_lower_case=True)
    model = GPT2LMHeadModel.from_pretrained(args.model_path)
    model.to(device)
    model.eval()
    print('Start product title to generate description, enter CTRL + Z to exit')
    try:
        while True:
            title = input("The product title entered is:")
            contents = predict_one_sample(model, tokenizer, device, args,
                                        title)
            for i, content in enumerate(contents):
                print("The {}th product description generated is：{}".format(i + 1, content))
    except:
        pass

In [17]:
#Product description generation test,You can change different checkpoints or inputs to get different generated test results
main_generate()

10/04/2022 10:48:46 - INFO - transformers.tokenization_utils_base -   Model name 'vocab/vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'vocab/vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
10/04/2022 10:48:46 - WARNING - transformers.tokenization_utils_base -   Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
10/04/2022 10:48:46 - I

Start product title to generate description, enter CTRL + Z to exit


The product title entered is: 苏打水


The 1th product description generated is：不易踩空摔跤，干爽透气，多种款式可供选择
The 2th product description generated is：新颖水牙线，全面顾及到牙齿各个角落，解决隐藏的污垢
The 3th product description generated is：新鲜现摘当季蔬菜，天然回甜味，优质的火锅食材


The product title entered is: 苏打水


The 1th product description generated is：采用优质工艺，完美的做工工艺，有赏心悦目之感，完美诠释出设计师的灵动写意，简单复古线条拥有经典的器型与独有的气质，充满了文艺典雅的格调
The 2th product description generated is：简约百搭风格，休闲感十足，有前卫时髦气质经典永不落伍，无论出街还是凹造型都很不错
The 3th product description generated is：简约的线条，优雅的绿植，彰显北欧轻奢风，品质保证挑细选，绿化居家环境


The product title entered is: 苏打水


The 1th product description generated is：优选好品质原料，个头饱满传统工艺，外形大颗颗颗饱满
The 2th product description generated is：采用铁艺支架的吸顶灯不生锈且防腐又永不褪色
The 3th product description generated is：经典又不落伍，马甲侧边开叉设计，拥有随性慵懒范


### 10.Translation Generated Corpora

In [ ]:
# Adapted from Youdao - https://ai.youdao.com/DOCSIRMA/html/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E7%BF%BB%E8%AF%91/API%E6%96%87%E6%A1%A3/%E6%96%87%E6%9C%AC%E7%BF%BB%E8%AF%91%E6%9C%8D%E5%8A%A1/%E6%96%87%E6%9C%AC%E7%BF%BB%E8%AF%91%E6%9C%8D%E5%8A%A1-API%E6%96%87%E6%A1%A3.html#p08
#Using a Chinese-English API, and you can copy the generated results for translation and view.


import json
import requests

# Translation function, ‘word’ is the content that needs to be translated
def translate(word):
    # Youdao dictionary API
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    key = {
        'type': "AUTO",
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
        "typoResult": "true"
    }
    
    response = requests.post(url, data=key)
    if response.status_code == 200:
        return response.text
    else:
        print("Youdao Dictionary failed to use")
        return None

def get_reuslt(repsonse):
    # Load the returned result into json format through json.loads
    result = json.loads(repsonse)
    print ("input：%s" % result['translateResult'][0][0]['src'])
    print ("output：%s" % result['translateResult'][0][0]['tgt'])

def main():
    word = input('Please enter the word or sentence you want to translate：')
    list_trans = translate(word)
    get_reuslt(list_trans)

if __name__ == '__main__':
    main()
    
#You can run this chapter to copy different generated corpus tests    
#The translation results may be inaccurate, for reference only, and will be explained in detail later.

**Note: This is the generated content when I enter the product title "Men's baseball uniform". Although the generated corpus looks very smooth and interesting, what I want is "baseball uniform", but the generated corpus It's "leather coat" related corpus**

### 11.Evaluate Results(BLEU)

At the beginning, I tried to use test_data, **30%** of train_data for evaluation, but the amount of data was too large to run, so I changed it to select the first **100 samples** for evaluation.

In [18]:
#Use BLEU to evaluate the differences between corpora generated by the model and top-quality existing samples.
from rouge import Rouge
import numpy as np

def compute_rouge(decoded_preds, decoded_labels):
    decoded_preds = [' '.join(list(hyp)) for hyp in decoded_preds]
    decoded_labels = [' '.join(list(ref)) for ref in decoded_labels]
    # print()
    rouge = Rouge()
    scores = []
    for i, j in zip(decoded_preds, decoded_labels):
        score = rouge.get_scores(i, j)
        scores.append([
                    score[0]['rouge-1']['f'], score[0]['rouge-2']['f'],
                    score[0]['rouge-l']['f']])
    rouge1 = np.mean([i[0] for i in scores])
    rouge2 = np.mean([i[1] for i in scores])
    rougel = np.mean([i[2] for i in scores])
    result = {"rouge1": round(rouge1, 4), "rouge2": round(rouge2, 4), "rouge-l": round(rougel, 4)}
    return result

def compute_bleu(hypothesis, reference):
    hypothesis = [' '.join(list(hyp)) for hyp in hypothesis]
    reference = [[' '.join(list(ref))] for ref in reference]
    return sacrebleu.corpus_bleu(hypothesis, reference)

def compute_test_bleu_and_rouge():
    args = set_args_generate()
    args.batch_size = 1
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICE"] = args.device
    device = torch.device("cuda" if torch.cuda.is_available()
                          and int(args.device) >= 0 else "cpu")
    tokenizer = BertTokenizer.from_pretrained(args.vocab_path,
                                              do_lower_case=True)
    model = GPT2LMHeadModel.from_pretrained(args.model_path)
    model.to(device)
    model.eval()
    hypothesis = []
    reference = []
    with open('data_dir/test.json',"r", encoding="utf-8") as fh:
            line = fh.readlines()
             #Change the number of test_Data samples here
            data = line[:100] 
            for line in tqdm(data, desc="iter",disable=False):
                line = line.strip()
                if not line:
                    continue
                sample = json.loads(line)
                content = sample['title']
                reference.append(sample['content'])
                titles = predict_one_sample(model, tokenizer, device, args,
                                            content)
                hypothesis.append(titles[0])
    bleu = compute_bleu(hypothesis, reference)
    rouge = compute_rouge(hypothesis, reference)    
    print(bleu)
    print(rouge)
    with open('bleu.txt','w',encoding='utf8') as f:
        f.write(f'{bleu}\n') #Save BLEU file
    with open('rouge.txt','w',encoding='utf8') as f:
        f.write(f'{rouge}\n') #Save BLEU file

In [19]:
# Calculate BLEU and Rouge
compute_test_bleu_and_rouge()

10/04/2022 10:49:40 - INFO - transformers.tokenization_utils_base -   Model name 'vocab/vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'vocab/vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
10/04/2022 10:49:40 - WARNING - transformers.tokenization_utils_base -   Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
10/04/2022 10:49:40 - I

BLEU = 4.70 53.8/4.0/2.1/1.1 (BP = 1.000 ratio = 1.000 hyp_len = 26 ref_len = 26)
{'rouge1': 0.036, 'rouge2': 0.0041, 'rouge-l': 0.0328}
